In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd
from collections import defaultdict
import random
import os
import gzip
import json
recommendation_path = '/content/drive/MyDrive/Capstone/output_batches/recommender_all_dedup.csv'
df_recommendations = pd.read_csv(recommendation_path)


df_interactions = pd.read_csv('/content/drive/MyDrive/Capstone/dataset/interaction_part.csv')

df_interactions.columns = df_interactions.columns.str.lower()

# indir = '/content/drive/MyDrive/Capstone/dataset/'


# filepath = os.path.join(indir, 'goodreads_interactions_comics_graphic.json.gz')
# lines = []
# with gzip.open(filepath, 'rt', encoding='utf-8') as f:
#     for line in f:
#         line = json.loads(line)
#         content = {
#             'user_id': line['user_id'],
#             'book_id': line['book_id'],
#             'rating': line['rating']
#         }
#         lines.append(content)


# df_interactions = pd.DataFrame.from_records(lines)
def evaluate_precision_tpr(df_recommendations, df_interactions, k=10, max_users=1000):
    user_books = defaultdict(set)
    for _, row in df_interactions.iterrows():
        user_books[row['user_id']].add(row['book_id'])

    total_hits = 0
    total_preds = 0
    total_truth = 0

    for book_id in df_recommendations['input_book_id'].unique():
        recommended_books = df_recommendations[
            df_recommendations['input_book_id'] == book_id
        ]['recommended_book_id'].tolist()[:k]

        users = [user for user, books in user_books.items() if book_id in books]
        if len(users) > max_users:
            users = random.sample(users, max_users)
        total_truth += len(users) * k
        for user in users:
            books = user_books[user] - {book_id}
            hits = sum([1 for r in recommended_books if r in books])
            total_hits += hits
            total_preds += k


    precision = total_hits / total_preds if total_preds > 0 else 0
    tpr = total_hits / total_truth if total_truth > 0 else 0
    return precision, tpr

precision, tpr = evaluate_precision_tpr(df_recommendations, df_interactions, k=10)
print(f"Content Model - Precision@10: {precision:.4f}")
print(f"Content Model - TPR@10: {tpr:.4f}")




<ipython-input-6-9d24aec28a40>:11: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_interactions = pd.read_csv('/content/drive/MyDrive/Capstone/dataset/interaction_part.csv')


Content Model - Precision@10: 0.2127
Content Model - TPR@10: 0.2127
